In [2]:
import numpy as np
from ortools.sat.python import cp_model

In [16]:
puzzle_size = 5

In [17]:
model = cp_model.CpModel()

In [18]:
grid = np.array([model.NewIntVar(1, 5, f'{i}') for i in range(puzzle_size ** 2)])

grid.reshape((puzzle_size, puzzle_size))

array([[0(1..5), 1(1..5), 2(1..5), 3(1..5), 4(1..5)],
       [5(1..5), 6(1..5), 7(1..5), 8(1..5), 9(1..5)],
       [10(1..5), 11(1..5), 12(1..5), 13(1..5), 14(1..5)],
       [15(1..5), 16(1..5), 17(1..5), 18(1..5), 19(1..5)],
       [20(1..5), 21(1..5), 22(1..5), 23(1..5), 24(1..5)]], dtype=object)

In [21]:
# need to add constraint to all rows and column -- all different
grid[0, :]

IndexError: too many indices for array